In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import cv2
import tensorflow as tf

### Preparing the Dataset

In [2]:
BATCH_SIZE = 64
IMG_SIZE   = (128, 128)

IMAGES_DIR  = '../DatasetCleaning/'
USAGE_DIRS  = [ 'croppedface_no_mask', 'croppedface_with_mask', 'croppedface_with_mask_incorrect' ]
IMG_LABEL_PAIRS = []

for dir in USAGE_DIRS:
    class_name      = dir[len('cropped') : ]
    IMG_LABEL_PAIRS = IMG_LABEL_PAIRS + [ ('{}/{}'.format(dir, x), class_name) for x in os.listdir(IMAGES_DIR + dir)]

images_df = pd.DataFrame(IMG_LABEL_PAIRS, columns = ['filename', 'class'])
image_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0 / 255)
train_ds  = image_gen.flow_from_dataframe(images_df, 
                                          directory   = IMAGES_DIR,
                                          x_col       = 'filename',
                                          y_col       = 'class',
                                          target_size = IMG_SIZE,
                                          batch_size  = BATCH_SIZE)

Found 5888 validated image filenames belonging to 3 classes.


### Face mask usage classifier

In [3]:
usage_model = tf.keras.models.Sequential([
    tf.keras.applications.ResNet50(include_top = False, input_shape = (*IMG_SIZE,3), pooling = 'max'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(3, activation = 'softmax'),
])
usage_model.compile(optimizer = 'adam', loss = tf.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
history = usage_model.fit(train_ds, epochs=10)